In [2]:
import cavro

# Options

The behaviour of `cavro` can be customized throught the [Options](/docs/api/#class-options) class.

Many of the options are not helpful to configure other than in special circumstances, however some may be useful for compatibility or useability reasons.

## Common options


### `record_encodes_to_dict`/`record_can_encode_dict`/`*record*`

See [the record type guide](/docs/user-guide/records)


### `allow_tuple_notation`

With this option set to `True`, values passed to encode that are a `tuple` with two items have a special meaning.
The tuple values are:

`(<type name>, <value>)`

cavro will use the type name to ensure that the value is encoded correctly.  For example:

In [27]:
schema = cavro.Schema(
    ['string', {'type': 'enum', 'name': 'X', 'symbols': ['AAAAA', 'BBBBB']}],
    allow_tuple_notation=True
    )

def round_trip(msg, value):
    encoded = schema.binary_encode(value)
    decoded = schema.binary_decode(encoded)
    print(f'{msg.ljust(13)}: {str(value).ljust(19)} → {str(encoded).ljust(15)} → {decoded}')

round_trip('Normal',       'AAAAA',)
round_trip('Tuple string', ('string', 'AAAAA'))
round_trip('Tuple enum', ('X', 'AAAAA'))


Normal       : AAAAA               → b'\x00\nAAAAA'  → AAAAA
Tuple string : ('string', 'AAAAA') → b'\x00\nAAAAA'  → AAAAA
Tuple enum   : ('X', 'AAAAA')      → b'\x02\x00'     → AAAAA


You can see that we end up with the same result each time, but the encoded value of the last tine is much shorter than the others, because the enum symbol has been encoded rather than a string.

The same applies to records:

In [35]:
schema = cavro.Schema(
    [
        {'type': 'record', 'name': 'X', 'fields': [{'name': 'a', 'type': 'int'}]},
        {'type': 'record', 'name': 'Y', 'fields': [{'name': 'a', 'type': 'int'}]},
    ],
    allow_tuple_notation=True
)

round_trip('Normal',  {'a': 1})
round_trip('Tuple X', ('X', {'a': 1}))
round_trip('Tuple Y', ('Y', {'a': 1}))

Normal       : {'a': 1}            → b'\x00\x02'     → <Record:X {a: 1}>
Tuple X      : ('X', {'a': 1})     → b'\x00\x02'     → <Record:X {a: 1}>
Tuple Y      : ('Y', {'a': 1})     → b'\x02\x02'     → <Record:Y {a: 1}>


### `fingerprint_returns_digest`

By default, the `Schema.fingerprint` method returns an object that is a `hashlib.HASH` object
allowing the caller to choose which representation of the hash to use.

If you set this to `True`, then `Schema.fingerprint` returns a binary digest of the value:

In [6]:
print(cavro.Schema('{"type": "string"}').fingerprint('sha256'))
print(cavro.Schema('{"type": "string"}', fingerprint_returns_digest=True).fingerprint('sha256'))

<sha256 _hashlib.HASH object @ 0x10a3e2e70>
b"\xe9\xe5\xc1\xc9\xe4\xf6's9\xd1\xbc\xde\x073\xa5\x9b\xd4/\x871\xf4I\xdam\xc10\x10\xa9\x16\x93\rH"
